In [91]:
!java -version

openjdk version "1.8.0_382"
OpenJDK Runtime Environment (build 1.8.0_382-8u382-ga-1~20.04.1-b05)
OpenJDK 64-Bit Server VM (build 25.382-b05, mixed mode)


In [47]:
!pip install pyspark==3.3.1

In [88]:
from pyspark.sql import SparkSession
import os
import subprocess

dst_lh_url  = os.environ.get('DST_LH_URL', 'spark://spark-master-0.spark-headless.demml.svc.cluster.local:7077')
dst_lh_appn = os.environ.get('DST_LH_APPN', 'iris_naive_bayes_classification_pyspark_EDU_6')
my_pod_ip = subprocess.run(['hostname', '-I'], stdout=subprocess.PIPE).stdout.decode('utf-8').strip(' \n\t')

spark = SparkSession.builder \
    .master(dst_lh_url) \
    .appName(dst_lh_appn) \
    .config('spark.jars.packages','org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.1.0,org.mariadb.jdbc:mariadb-java-client:2.3.0,org.postgresql:postgresql:42.5.4,software.amazon.awssdk:bundle:2.17.257,software.amazon.awssdk:url-connection-client:2.17.257,software.amazon.awssdk:s3:2.17.257,software.amazon.awssdk:iam:2.17.257,org.apache.hadoop:hadoop-aws:3.2.3') \
    .config('spark.driver.host', my_pod_ip) \
    .config('spark.scheduler.mode', 'FAIR') \
    .getOrCreate()
spark.sparkContext.setLogLevel("WARN")
spark

<pyspark.sql.session.SparkSession object at 0x7f3fff879070>

In [89]:
df = spark.read.csv("/home/coder/eromero/iris-pyspark/data/01_raw/iris.csv", header=True, inferSchema=True)

23/09/13 12:59:21 WARN TaskSetManager: Lost task 0.0 in stage 2.0 (TID 8) (172.21.23.198 executor 0): java.io.FileNotFoundException: 
File file:/home/coder/eromero/iris-pyspark/data/01_raw/iris.csv does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:661)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:212)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:270)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.s

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 df = spark.read.csv("/home/coder/eromero/iris-pyspark/data/01_raw/iris.csv", header=True     │
│   2                                                                                              │
│                                                                                                  │
│ /home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/pyspark/sql/readwriter.py:535 in   │
│ csv                                                                                              │
│                                                                                                  │
│    532 │   │   │   path = [path]                                                                 │
│    533 │   │   if type(path) == list:                                                            │
│    534 │   │   │   assert self._spark._sc._jvm is not None                                       │
│ ❱  535 │   │   │   return self._df(self._jreader.csv(self._spark._sc._jvm.PythonUtils.toSeq(pat  │
│    536 │   │   elif isinstance(path, RDD):                                                       │
│    537 │   │   │                                                                                 │
│    538 │   │   │   def func(iterator):                                                           │
│                                                                                                  │
│ /home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/py4j/java_gateway.py:1321 in       │
│ __call__                                                                                         │
│                                                                                                  │
│   1318 │   │   │   proto.END_COMMAND_PART                                                        │
│   1319 │   │                                                                                     │
│   1320 │   │   answer = self.gateway_client.send_command(command)                                │
│ ❱ 1321 │   │   return_value = get_return_value(                                                  │
│   1322 │   │   │   answer, self.gateway_client, self.target_id, self.name)                       │
│   1323 │   │                                                                                     │
│   1324 │   │   for temp_arg in temp_args:                                                        │
│                                                                                                  │
│ /home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/pyspark/sql/utils.py:190 in deco   │
│                                                                                                  │
│   187 def capture_sql_exception(f: Callable[..., Any]) -> Callable[..., Any]:                    │
│   188 │   def deco(*a: Any, **kw: Any) -> Any:                                                   │
│   189 │   │   try:                                                                               │
│ ❱ 190 │   │   │   return f(*a, **kw)                                                             │
│   191 │   │   except Py4JJavaError as e:                                                         │
│   192 │   │   │   converted = convert_exception(e.java_exception)                                │
│   193 │   │   │   if not isinstance(converted, UnknownException):                                │
│                                                                                                  │
│ /home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/py4j/protocol.py:326 in            │
│ get_return_value                                           

In [90]:
spark.stop()

In [81]:
%reload_kedro
catalog.list()

[09/13/23 12:40:39] INFO     Resolved project path as: /home/coder/eromero/iris-pyspark.            ]8;id=164348;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=77961;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py#139\139]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

23/09/13 12:40:40 WARN FairSchedulableBuilder: Fair Scheduler configuration file not found so jobs will be scheduled in FIFO order. To use fair scheduling, configure pools in fairscheduler.xml or set spark.scheduler.allocation.file to a file that contains the configuration.


[09/13/23 12:40:41] INFO     Kedro project iris-pyspark                                             ]8;id=650917;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=126312;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py#108\108]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=775367;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=107727;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py#109\109]8;;\
                             'pipelines'                                                                           

[09/13/23 12:40:47] INFO     Registered line magic 'run_viz'                                        ]8;id=889086;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=906677;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py#115\115]8;;\

['iris_local', 'parameters', 'params:example_test_data_ratio', 'params:example_num_trees']

In [82]:
iris_local = catalog.load('iris_local')

[09/13/23 12:40:50] INFO     Loading data from 'iris_local' (SparkDataSet)...                   ]8;id=75850;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=371578;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/io/data_catalog.py#492\492]8;;\

23/09/13 12:40:52 WARN TaskSetManager: Lost task 0.0 in stage 0.0 (TID 0) (172.21.23.198 executor 0): java.io.FileNotFoundException: 
File file:/home/coder/eromero/iris-pyspark/data/01_raw/iris.csv does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:661)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:212)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:270)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.s

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/io/core.py:192 in load       │
│                                                                                                  │
│   189 │   │   self._logger.debug("Loading %s", str(self))                                        │
│   190 │   │                                                                                      │
│   191 │   │   try:                                                                               │
│ ❱ 192 │   │   │   return self._load()                                                            │
│   193 │   │   except DatasetError:                                                               │
│   194 │   │   │   raise                                                                          │
│   195 │   │   except Exception as exc:                                                           │
│                                                                                                  │
│ /home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro_datasets/spark/spark_dataset │
│ .py:397 in _load                                                                                 │
│                                                                                                  │
│   394 │   │   if self._schema:                                                                   │
│   395 │   │   │   read_obj = read_obj.schema(self._schema)                                       │
│   396 │   │                                                                                      │
│ ❱ 397 │   │   return read_obj.load(load_path, self._file_format, **self._load_args)              │
│   398 │                                                                                          │
│   399 │   def _save(self, data: DataFrame) -> None:                                              │
│   400 │   │   save_path = _strip_dbfs_prefix(self._fs_prefix + str(self._get_save_path()))       │
│                                                                                                  │
│ /home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/pyspark/sql/readwriter.py:177 in   │
│ load                                                                                             │
│                                                                                                  │
│    174 │   │   │   self.schema(schema)                                                           │
│    175 │   │   self.options(**options)                                                           │
│    176 │   │   if isinstance(path, str):                                                         │
│ ❱  177 │   │   │   return self._df(self._jreader.load(path))                                     │
│    178 │   │   elif path is not None:                                                            │
│    179 │   │   │   if type(path) != list:                                                        │
│    180 │   │   │   │   path = [path]  # type: ignore[list-item]                                  │
│                                                                                                  │
│ /home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/py4j/java_gateway.py:1321 in       │
│ __call__                                                                                         │
│                                                                                                  │
│   1318 │   │   │   proto.END_COMMAND_PART                                                        │
│   1319 │   │                                                                                     │
│   1320 │   │   answer = self.gateway_client.send_command(command)                                │
│ ❱ 1321 │   │   return_value = get_return_value(            

In [86]:
iris_local.show()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 iris_local.show()                                                                            │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'iris_local' is not defined